<div dir="rtl" style="line-height: 1.9">
<h1>عرفان شیرمحمدی</h1>
</div>

In [1]:
import cv2
import numpy as np
import IPython

<div dir="rtl" style="line-height: 1.9">
  <h1>Mean Shift </h1>
  <p>الگوریتم شیفت میانگین یک تکنیک خوشه بندی غیر پارامتری است که برای شناسایی
    خوشه هایی از داده(نقاط) برای یک مجموعه بدون برچشب استفاده میشود</p>
  <p>با تغییر مکرر به سمت مناطق با چگالی بالاتر نقاط ، این الگوریتم خوشه‌های داده
    را بدون نیاز به فرضیات ایجاد می‌کند.</p>
  <p>طرض کار این الگوریتم به این صورت است که فرض میکنیم مجموعه از نقاط داده که میتوند توزیعی از پیکسل ها و یا هیستوگرام
    باشد داریم و پنجره ای داریم که باید این پنجره را به ناحیه ای که حداکثر تراکم پیکسلی یا نقاط را دارد منتقل کنیم مانند
    تصویر زیر :</p>

  <p style="text-align:center"><img src="https://docs.opencv.org/4.10.0/meanshift_basics.jpg" /></p>

  <p>الگوریتم Mean Shift الگوریتم تپه نوردی محسوب میشود که از تریغ شیفت دادن مکرر کرنل به ناحیه ای با تراکم بالاتر داده
    تا زمان همگرایی سنتروید و مرکز چنجره است </p>

  <p>هر تغییر با یک بردار تغییر میانگین تعریف می شود. بردار تغییر میانگین همیشه به سمت حداکثر افزایش چگالی اشاره می کند
  </p>
  <p style="text-align:center"><img src="https://upload.wikimedia.org/wikipedia/commons/b/bd/Meanshiftred.gif"></p>

  <p>نقاط قوت</p>
  <li>انعطاف پذیر : به خوشه از پیش تعریف شده اشکال یا اعداد نیاز ندارد</li>
  <li>خوشه ها به هر شگلی میتوانند باشند</li>
  <li>با دیتا نویزی خوب کار میکند</li>
  <p>نقاط ضعف</p>
  <li>محاسبات سنگین</li>
  <li>نسبت به شکل خوشه حساس است</li>

</div>

In [2]:
cap = cv2.VideoCapture("../data/car.mp4")

# take first frame of the video
ret,frame = cap.read()

# Get the height and width of the frame (required to be an interger)
width = int(cap.get(3)) 
height = int(cap.get(4))

# Define the codec and create VideoWriter object. The output is stored in '*.avi' file.
out = cv2.VideoWriter('../data/car_meanshift.avi', cv2.VideoWriter_fourcc(*"MJPG"), 30, (width, height))

# setup initial location of window
y,h,x,w = 320,90,600,125  # simply hardcoded the values
track_window = (x,y,w,h)

# set up the ROI for tracking
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = None
# mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret, frame = cap.read()

    if ret is True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,255),2)
        out.write(img2)
        #imshow('Tracking', img2)

    else:
        break

cap.release()
out.release()

In [4]:
IPython.display.Video(url="../data/car racing.mp4")

<div dir="rtl" style="line-height: 1.9">
<h1>Cam Shift</h1>
<p>این الگوریتم بسط داده شده شیفت میانگین است MeanShift از یک پنجره جستجو با اندازه ثابت استفاده می کند، در حالی که CamShift از یک پنجره جستجوی  پویا استفاده می کند که می تواند با تغییرات در اندازه و شکل شی تطبیق یابد.</p>
<p style="text-align:center"><img src="https://upload.wikimedia.org/wikipedia/commons/8/86/CamshiftStillImage.gif"></p>
</div>

In [ ]:
cap = cv2.VideoCapture("../data/car.mp4")

# take first frame of the video
ret,frame = cap.read()

# Get the height and width of the frame (required to be an interger)
width = int(cap.get(3)) 
height = int(cap.get(4))

# Define the codec and create VideoWriter object. The output is stored in '*.avi' file.
out = cv2.VideoWriter('../data/car_camshift.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

# setup initial location of window
y,h,x,w = 250,90,400,125  # simply hardcoded the values
track_window = (x,y,w,h)

# set up the ROI for tracking
roi = frame[y:y+h, x:x-w]
hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = None
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret ,frame = cap.read()

    if ret is True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        # Draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.intp(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        out.write(img2)
        #imshow('img2',img2)

    else:
        break

cap.release()
out.release()